#### The goal of this exercise is to design a route for each vehicle so that all customers are served, and the number of vehicles

#### 1. (objective 1) along with the total traveled distance.
#### 2. (objective 2) by all the vehicles are minimized. In addition, the capacity of each vehicle should not be exceeded (constraint 1).

### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import re
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

### Reading file

In [42]:
with open(r'Input_Data.txt') as data:
    lines = data.readlines()
    
lines = [i.split('\n')[0] for i in lines]
lines

['C104',
 '',
 'VEHICLE',
 'NUMBER     CAPACITY',
 '  10         70',
 '',
 'CUSTOMER',
 'CUST NO.   XCOORD.  YCOORD.    DEMAND    READY TIME  DUE DATE  SERVICE   TIME',
 ' ',
 '    0      40         50          0          0       1236          0   ',
 '    1      45         68         10          0       1127         90   ',
 '    2      45         70         30          0       1125         90   ',
 '    3      42         66         10          0       1129         90   ',
 '    4      42         68         10        727        782         90   ',
 '    5      42         65         10          0       1130         90   ',
 '    6      40         69         20          0       1127         90   ',
 '    7      40         66         20          0       1130         90   ',
 '    8      38         68         20        255        324         90   ',
 '    9      38         70         10        534        605         90   ',
 '   10      35         66         10          0       1129     

### Creating a dataframe from file

In [43]:
test_data=[re.findall('[0-9]+',i)for i in lines[9:-1]]


In [44]:
test_data=pd.DataFrame(test_data,columns = ['cust_no','XCOORD','YCOORD','DEMAND','READY TIME','DUE DATE','TIME'])
test_data = test_data.astype('float')
test_data

,cust_no,XCOORD,YCOORD,DEMAND,READY TIME,DUE DATE,TIME
0,0.0,40.0,50.0,0.0,0.0,1236.0,0.0
1,1.0,45.0,68.0,10.0,0.0,1127.0,90.0
2,2.0,45.0,70.0,30.0,0.0,1125.0,90.0
3,3.0,42.0,66.0,10.0,0.0,1129.0,90.0
4,4.0,42.0,68.0,10.0,727.0,782.0,90.0
5,5.0,42.0,65.0,10.0,0.0,1130.0,90.0
6,6.0,40.0,69.0,20.0,0.0,1127.0,90.0
7,7.0,40.0,66.0,20.0,0.0,1130.0,90.0
8,8.0,38.0,68.0,20.0,255.0,324.0,90.0
9,9.0,38.0,70.0,10.0,534.0,605.0,90.0


In [29]:
## We have the following information provided :
max_vehicles = 10                                     # Number of vehicles that can be used
cap = 70                                              #capacity per vehicle
customers = [i for i in range(1,26)]                  # Total number of customers to be served
total_nodes = [0]+customers                           # Total nodes (depot + 25 customers)
total_paths = [(i,j) for i in total_nodes for j in total_nodes if i!=j] # Total possible combination of traversals between nodes 


In [50]:
## Calculation of distance between all traversals using eucildean method

distance = {(i,j):np.hypot(test_data.XCOORD.iloc[i]-test_data.XCOORD.iloc[j],test_data.YCOORD.iloc[i]-test_data.YCOORD.iloc[j])for i,j in total_paths}

In [52]:
## Creation of distance matrix to keep track of distance w.r.t each node traversal


distance_matrix = []
for i in range(26):
    inner = []
    for j in range(26):
        if i == j:
            inner.append(0)
        else:
            inner.append(distance[i,j])
    distance_matrix.append(inner)

In [54]:
## Structuring of dictionary using function which would create the input for our CVRP model

def create_data_model():
    """Stores the data for the problem."""
    data = {}   
    data['distance_matrix'] = d                  ## Distance matrix containing Euclidean distances between nodes
    data['demands'] = list(test_data.DEMAND)     ## Demands of each customer
    data['vehicle_capacities'] = [70 for i in range(10)] ## Vehicle capacity for each vehicle which is 70 for each vehicle
    data['num_vehicles'] = 10  ## Number of vehicles which is 10
    data['depot'] = 0 ## Depot is node 0
    return data

In [55]:
data= create_data_model()
data

{'distance_matrix': [[0,
   18.681541692269406,
   20.615528128088304,
   16.1245154965971,
   18.110770276274835,
   15.132745950421556,
   19.0,
   16.0,
   18.110770276274835,
   20.09975124224178,
   16.76305461424021,
   19.6468827043885,
   38.07886552931954,
   30.805843601498726,
   39.35733730830886,
   36.05551275463989,
   40.311288741492746,
   33.301651610693426,
   35.35533905932738,
   39.05124837953327,
   10.0,
   10.198039027185569,
   12.165525060596439,
   13.0,
   15.0,
   15.132745950421556],
  [18.681541692269406,
   0,
   2.0,
   3.605551275463989,
   3.0,
   4.242640687119285,
   5.0990195135927845,
   5.385164807134504,
   7.0,
   7.280109889280518,
   10.198039027185569,
   10.04987562112089,
   26.248809496813376,
   24.041630560342615,
   28.600699292150182,
   27.730849247724095,
   30.23243291566195,
   27.892651361962706,
   30.805843601498726,
   32.31098884280702,
   23.430749027719962,
   21.93171219946131,
   23.345235059857504,
   21.400934559032695

In [56]:
# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

In [57]:
# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

### Running the model for the inputs

In [58]:
def main(data=data):
    """Entry point of the program."""
    # Instantiate the data problem.
    data = data

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        3000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print('No solution found !')


if __name__ == '__main__':
    main()

Objective: 8309
Route for vehicle 0:
 0 ->  7 ->  5 ->  3 ->  1 ->  2 ->  4 ->  6 ->  9 ->  8 ->  10 ->  23 ->  25 ->  24 ->  22 ->  21 ->  20 -> 0
Distance of the route: 72m

Route for vehicle 1:
 0 ->  18 ->  19 ->  17 -> 0
Distance of the route: 78m

Route for vehicle 2:
 0 ->  13 ->  15 ->  16 -> 0
Distance of the route: 80m

Route for vehicle 3:
 0 ->  11 ->  12 ->  14 -> 0
Distance of the route: 79m

Route for vehicle 4:
 0 -> 0
Distance of the route: 0m

Route for vehicle 5:
 0 -> 0
Distance of the route: 0m

Route for vehicle 6:
 0 -> 0
Distance of the route: 0m

Route for vehicle 7:
 0 -> 0
Distance of the route: 0m

Route for vehicle 8:
 0 -> 0
Distance of the route: 0m

Route for vehicle 9:
 0 -> 0
Distance of the route: 0m

Maximum of the route distances: 80m


### Summary :

1. Only 3 vehicles out of 10 were used to complete demands for 25 customers.
2. Path traversed by vehicles 1,2,3 are stated above.